## DimUser

In [0]:
print("Mamba Mentality")
print("Data Engineering is not about handling messy datasets.It's about building the solutions")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
import os
import sys

project_path = os.path.join(os.getcwd(), '..','..')
sys.path.append(project_path)
project_path

from utils.transformations import reusable

## Auto Loader

In [0]:
df_user = spark.readStream.format("cloudFiles")\
                .option("cloudFiles.format" , "parquet")\
                .option("cloudFiles.schemaLocation","abfss://silver@nithindev.dfs.core.windows.net/DimUser/checkpoint")\
                .option("schemaEvolution","addNewColumns")\
                .load("abfss://bronze@nithindev.dfs.core.windows.net/DimUser")

In [0]:
df_transformation = df_user.withColumn("user_name",upper(col("user_name")))
df_transformation = df_user.dropDuplicates(["user_id"])
df_transformation = reusable().dropColumns(df_user,["_rescued_data"])
display(df_transformation)

In [0]:
df_transformation.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@nithindev.dfs.core.windows.net/DimUser/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@nithindev.dfs.core.windows.net/DimUser/data")\
    .toTable("spotify_data.silver.dimUser")

## DimArtist

In [0]:
df_artist = spark.readStream.format("cloudFiles")\
                .option("cloudFiles.format" , "parquet")\
                .option("cloudFiles.schemaLocation","abfss://silver@nithindev.dfs.core.windows.net/DimArtist/checkpoint")\
                .option("schemaEvolution","addNewColumns")\
                .load("abfss://bronze@nithindev.dfs.core.windows.net/DimArtist")
display(df_artist)

In [0]:
df_art_obj = reusable()

df_artist = df_art_obj.dropColumns(df_artist,["_rescued_data"])
df_artist = df_artist.dropDuplicates(["artist_id"])
display(df_artist)

In [0]:
df_artist.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@nithindev.dfs.core.windows.net/DimArtist/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@nithindev.dfs.core.windows.net/DimArtist/data")\
    .toTable("spotify_data.silver.dimArtist")

## DimTrack

In [0]:
df_track = spark.readStream.format("cloudFiles")\
                .option("cloudFiles.format" , "parquet")\
                .option("cloudFiles.schemaLocation","abfss://silver@nithindev.dfs.core.windows.net/DimTrack/checkpoint")\
                .option("schemaEvolution","addNewColumns")\
                .load("abfss://bronze@nithindev.dfs.core.windows.net/DimTrack")
display(df_track)

In [0]:
df_track = df_track.withColumn("durationFlag" , when(col("duration_sec") < 150 , 'Low')\
                                                .when(col('duration_sec') < 300 , 'Medium')\
                                                .otherwise('High'))
df_track = df_track.withColumn("track_name",regexp_replace(col('track_name'),'-',' '))

df_track = reusable().dropColumns(df_track,["_rescued_data"])
display(df_track)


In [0]:
df_track.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@nithindev.dfs.core.windows.net/DimTrack/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@nithindev.dfs.core.windows.net/DimTrack/data")\
    .toTable("spotify_data.silver.dimTrack")

## DimDate

In [0]:
df_date = spark.readStream.format("cloudFiles")\
                .option("cloudFiles.format" , "parquet")\
                .option("cloudFiles.schemaLocation","abfss://silver@nithindev.dfs.core.windows.net/DimDate/checkpoint")\
                .option("schemaEvolution","addNewColumns")\
                .load("abfss://bronze@nithindev.dfs.core.windows.net/DimDate")
display(df_date)

In [0]:
df_date = reusable().dropColumns(df_date,["_rescued_data"])

df_date.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@nithindev.dfs.core.windows.net/DimDate/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@nithindev.dfs.core.windows.net/DimDate/data")\
    .toTable("spotify_data.silver.dimDate")

## FactStream

In [0]:
df_fact = spark.readStream.format("cloudFiles")\
                .option("cloudFiles.format" , "parquet")\
                .option("cloudFiles.schemaLocation","abfss://silver@nithindev.dfs.core.windows.net/FactStream/checkpoint")\
                .option("schemaEvolution","addNewColumns")\
                .load("abfss://bronze@nithindev.dfs.core.windows.net/FactStream")
display(df_fact)

In [0]:
df_fact = reusable().dropColumns(df_fact,["_rescued_data"])

df_fact.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@nithindev.dfs.core.windows.net/FactStream/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@nithindev.dfs.core.windows.net/FactStream/data")\
    .toTable("spotify_data.silver.factStream")